In [1]:
from bs4 import BeautifulSoup
from fnmatch import fnmatch
from PIL import Image
from PIL import ImageFile
from pathlib import Path
from shutil import copyfile

import imagehash
import mysql.connector
import os
import re

In [2]:
# GENERAL CONFIGS

ImageFile.LOAD_TRUNCATED_IMAGES = True

# == Database config ==
conn = mysql.connector.connect(
  host="localhost",
  user="root",
  password="root"
)

cursor = conn.cursor()

DB_NAME = "blp_dataset"
TABLE_NAME = "products_tmp_area_51"


# == Files config ==
MAIN_FOLDER = r"D:\dnmarchives\novo\area51"

In [3]:
# Creating environment
cursor.execute("CREATE DATABASE IF NOT EXISTS {} DEFAULT CHARACTER SET 'utf8'".format(DB_NAME))
cursor.execute("USE {}".format(DB_NAME))

In [4]:
table_schema = (
    "CREATE TABLE IF NOT EXISTS products_tmp_area_51 ("
    "  id INT(11) AUTO_INCREMENT PRIMARY KEY NOT NULL,"
    "  name TEXT NOT NULL,"
    "  description TEXT,"
    "  price VARCHAR(50),"
    "  market_name VARCHAR(255),"
    "  seller_name TEXT,"
    "  has_image VARCHAR(25) NOT NULL DEFAULT 'has_image',"
    "  illegal ENUM('t', 'f') NOT NULL DEFAULT 't'"
    ") ENGINE=InnoDB")

cursor.execute(table_schema)

In [5]:
def update_image(product_id, value):
    query = "UPDATE products_tmp_area_51 SET has_image = %s WHERE id = %s"
    cursor.execute(query, (value, product_id))
    conn.commit()

In [6]:
def get_content(content):
    clear_content = ""

    if content is not None:
        raw_c = content.text

        clear_content = re.sub(r'[\t\r\n]', ' ', raw_c)
        clear_content = re.sub(' +',' ', clear_content).lstrip()
        
    return clear_content

In [ ]:
folders = [x for x in Path(MAIN_FOLDER).iterdir() if x.is_dir()]

for fo in folders:
    folder = Path(f"{fo}\shop")
    folder_image = Path(f"{fo}")

    if folder.exists():
        for file in os.listdir(folder):
            page_content = BeautifulSoup(open(Path(f"{folder}\{file}"), encoding='utf8').read())

            if page_content is not None:
                ## GET DESCRIPTION
                description = ""
                raw_desc = page_content.find("li", attrs={"id": "productDescription"})
                
                if raw_desc is not None:
                    raw_desc.find("h2").decompose()

                    if raw_desc is not None:
                        description = get_content(raw_desc)
                
                ## GET TITLE
                title = ""
                raw_title = page_content.find("li", attrs={"id": "productTitle"})
                
                if raw_title is not None:
                    title = get_content(raw_title)

                ## GET SELLER
                seller = ""
                raw_seller = page_content.find("div", attrs={"id": "vendorViewWrapper"})
                
                if raw_seller is not None:
                    raw_seller = raw_seller.find("a")
                    
                    if raw_seller is not None:
                        seller = get_content(raw_seller)
                
                
                if description != "" or title != "":
                    # Insert product into database
                    query = "INSERT INTO products_tmp_area_51 (name, description, market_name, seller_name, illegal) VALUES (%s, %s, %s, %s, %s)"
                    values = (title, description, "Armory", seller, "t")
                    
                    cursor.execute(query, values)
                    conn.commit()

                    product_id = cursor.lastrowid

                    ## GET IMAGE
                    img_content = page_content.find("li", attrs={"id": "productThumb"})
                    
                    if img_content is not None:
                        img_content = img_content.find("a", attrs={"class", "lightbox"})
                        
                        if img_content is not None:
                            img = img_content.find("img")["src"]
                            img_path = Path(f"{folder_image}\{img}")
                            
                            if img_path.exists():
                                try:
                                    # copy images to new directory, renaming with product id
                                    print(f"Image {product_id} exists... creating")

                                    dst_dir = Path(f"D:\images\p_area_51\{product_id}.jpg")
                                    copyfile(img_path, dst_dir)
                                except:
                                    print(f"Image {product_id} has error. Except")
                                    update_image(product_id, "no_image")
                            else:
                                print(f"Product {product_id} path image not found")
                                update_image(product_id, "no_image")
                        else:
                            print(f"Product {product_id} path image not found")
                            update_image(product_id, "no_image")  
                    else:
                        print(f"Product {product_id} has no image")
                        update_image(product_id, "no_image")

Image 1 exists... creating
Product 2 path image not found
Image 3 exists... creating
Image 4 exists... creating
Image 5 exists... creating
Image 6 exists... creating
Image 7 exists... creating
Image 7 has error. Except
Image 8 exists... creating
Image 9 exists... creating
Image 10 exists... creating
Image 10 has error. Except
Image 11 exists... creating
Image 12 exists... creating
Image 13 exists... creating
Image 14 exists... creating
Image 15 exists... creating
Image 16 exists... creating
Image 16 has error. Except
Image 17 exists... creating
Image 17 has error. Except
Image 18 exists... creating
Image 18 has error. Except
Image 19 exists... creating
Image 19 has error. Except
Image 20 exists... creating
Image 21 exists... creating
Image 22 exists... creating
Image 23 exists... creating
Image 24 exists... creating
Image 25 exists... creating
Image 26 exists... creating
Image 27 exists... creating
Image 28 exists... creating
Image 29 exists... creating
Image 30 exists... creating
Imag

Product 323 has no image
Product 324 has no image
Product 325 has no image
Product 326 has no image
Product 327 has no image
Product 328 has no image
Product 329 has no image
Product 330 has no image
Product 331 has no image
Product 332 has no image
Product 333 has no image
Product 334 has no image
Product 335 has no image
Product 336 has no image
Product 337 has no image
Product 338 has no image
Product 339 has no image
Product 340 has no image
Product 341 has no image
Product 342 has no image
Product 343 has no image
Product 344 has no image
Product 345 has no image
Product 346 has no image
Product 347 has no image
Product 348 has no image
Product 349 has no image
Product 350 has no image
Product 351 has no image
Product 352 has no image
Product 353 has no image
Product 354 has no image
Product 355 has no image
Product 356 has no image
Product 357 has no image
Product 358 has no image
Product 359 has no image
Product 360 has no image
Product 361 has no image
Product 362 has no image


Product 653 has no image
Product 654 has no image
Product 655 has no image
Product 656 has no image
Product 657 has no image
Product 658 has no image
Product 659 has no image
Product 660 has no image
Product 661 has no image
Product 662 has no image
Product 663 has no image
Product 664 has no image
Product 665 has no image
Product 666 has no image
Product 667 has no image
Product 668 has no image
Product 669 has no image
Product 670 has no image
Product 671 has no image
Product 672 has no image
Product 673 has no image
Product 674 has no image
Product 675 has no image
Product 676 has no image
Product 677 has no image
Product 678 has no image
Product 679 has no image
Product 680 has no image
Product 681 has no image
Product 682 has no image
Product 683 has no image
Product 684 has no image
Product 685 has no image
Product 686 has no image
Product 687 has no image
Product 688 has no image
Product 689 has no image
Product 690 has no image
Product 691 has no image
Product 692 has no image


Product 982 has no image
Product 983 has no image
Product 984 has no image
Product 985 has no image
Product 986 has no image
Product 987 has no image
Product 988 has no image
Product 989 has no image
Product 990 has no image
Product 991 has no image
Product 992 has no image
Product 993 has no image
Product 994 has no image
Product 995 has no image
Product 996 has no image
Product 997 has no image
Product 998 has no image
Product 999 has no image
Product 1000 has no image
Product 1001 has no image
Product 1002 has no image
Product 1003 has no image
Product 1004 has no image
Product 1005 has no image
Product 1006 has no image
Product 1007 has no image
Product 1008 has no image
Product 1009 has no image
Product 1010 has no image
Product 1011 has no image
Product 1012 has no image
Product 1013 has no image
Product 1014 has no image
Product 1015 has no image
Product 1016 has no image
Product 1017 has no image
Product 1018 has no image
Product 1019 has no image
Product 1020 has no image
Prod

Product 1302 has no image
Product 1303 has no image
Product 1304 has no image
Product 1305 has no image
Product 1306 has no image
Product 1307 has no image
Product 1308 has no image
Product 1309 has no image
Product 1310 has no image
Product 1311 has no image
Product 1312 has no image
Product 1313 has no image
Product 1314 has no image
Product 1315 has no image
Product 1316 has no image
Product 1317 has no image
Product 1318 has no image
Product 1319 has no image
Product 1320 has no image
Product 1321 has no image
Product 1322 has no image
Product 1323 has no image
Product 1324 has no image
Product 1325 has no image
Product 1326 has no image
Product 1327 has no image
Product 1328 has no image
Product 1329 has no image
Product 1330 has no image
Product 1331 has no image
Product 1332 has no image
Product 1333 has no image
Product 1334 has no image
Product 1335 has no image
Product 1336 has no image
Product 1337 has no image
Product 1338 has no image
Product 1339 has no image
Product 1340

...

Product 13573 has no image
Product 13574 has no image
Product 13575 has no image
Product 13576 has no image
Product 13577 has no image
Product 13578 has no image
Product 13579 has no image
Product 13580 has no image
Product 13581 has no image
Product 13582 has no image
Product 13583 has no image
Product 13584 has no image
Product 13585 has no image
Product 13586 has no image
Product 13587 has no image
Product 13588 has no image
Product 13589 has no image
Product 13590 has no image
Product 13591 has no image
Product 13592 has no image
Product 13593 has no image
Product 13594 has no image
Product 13595 has no image
Product 13596 has no image
Product 13597 has no image
Product 13598 has no image
Product 13599 has no image
Product 13600 has no image
Product 13601 has no image
Product 13602 has no image
Product 13603 has no image
Product 13604 has no image
Product 13605 has no image
Product 13606 has no image
Product 13607 has no image
Product 13608 has no image
Product 13609 has no image
P

Product 13880 has no image
Product 13881 has no image
Product 13882 has no image
Product 13883 has no image
Product 13884 has no image
Product 13885 has no image
Product 13886 has no image
Product 13887 has no image
Product 13888 has no image
Product 13889 has no image
Product 13890 has no image
Product 13891 has no image
Product 13892 has no image
Product 13893 has no image
Product 13894 has no image
Product 13895 has no image
Product 13896 has no image
Product 13897 has no image
Product 13898 has no image
Product 13899 has no image
Product 13900 has no image
Product 13901 has no image
Product 13902 has no image
Product 13903 has no image
Product 13904 has no image
Product 13905 has no image
Product 13906 has no image
Product 13907 has no image
Product 13908 has no image
Product 13909 has no image
Product 13910 has no image
Product 13911 has no image
Product 13912 has no image
Product 13913 has no image
Product 13914 has no image
Product 13915 has no image
Product 13916 has no image
P

Product 14188 has no image
Product 14189 has no image
Product 14190 has no image
Product 14191 has no image
Product 14192 has no image
Product 14193 has no image
Product 14194 has no image
Product 14195 has no image
Product 14196 has no image
Product 14197 has no image
Product 14198 has no image
Product 14199 has no image
Product 14200 has no image
Product 14201 has no image
Product 14202 has no image
Product 14203 has no image
Product 14204 has no image
Product 14205 has no image
Product 14206 has no image
Product 14207 has no image
Product 14208 has no image
Product 14209 has no image
Product 14210 has no image
Product 14211 has no image
Product 14212 has no image
Product 14213 has no image
Product 14214 has no image
Product 14215 has no image
Product 14216 has no image
Product 14217 has no image
Product 14218 has no image
Product 14219 has no image
Product 14220 has no image
Product 14221 has no image
Product 14222 has no image
Product 14223 has no image
Product 14224 has no image
P

Product 14492 has no image
Product 14493 has no image
Product 14494 has no image
Product 14495 has no image
Product 14496 has no image
Product 14497 has no image
Product 14498 has no image
Product 14499 has no image
Product 14500 has no image
Product 14501 has no image
Product 14502 has no image
Product 14503 has no image
Product 14504 has no image
Product 14505 has no image
Product 14506 has no image
Product 14507 has no image
Product 14508 has no image
Product 14509 has no image
Product 14510 has no image
Product 14511 has no image
Product 14512 has no image
Product 14513 has no image
Product 14514 has no image
Product 14515 has no image
Product 14516 has no image
Product 14517 has no image
Product 14518 has no image
Product 14519 has no image
Product 14520 has no image
Product 14521 has no image
Product 14522 has no image
Product 14523 has no image
Product 14524 has no image
Product 14525 has no image
Product 14526 has no image
Product 14527 has no image
Product 14528 has no image
P

Product 14797 has no image
Product 14798 has no image
Product 14799 has no image
Product 14800 has no image
Product 14801 has no image
Product 14802 has no image
Product 14803 has no image
Product 14804 has no image
Product 14805 has no image
Product 14806 has no image
Product 14807 has no image
Product 14808 has no image
Product 14809 has no image
Product 14810 has no image
Product 14811 has no image
Product 14812 has no image
Product 14813 has no image
Product 14814 has no image
Product 14815 has no image
Product 14816 has no image
Product 14817 has no image
Product 14818 has no image
Product 14819 has no image
Product 14820 has no image
Product 14821 has no image
Product 14822 has no image
Product 14823 has no image
Product 14824 has no image
Product 14825 has no image
Product 14826 has no image
Product 14827 has no image
Product 14828 has no image
Product 14829 has no image
Product 14830 has no image
Product 14831 has no image
Product 14832 has no image
Product 14833 has no image
P